In [1]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
import tensorflow as tf

In [3]:
# Loading the Trained Model, One Hot Encoding, Scaler

model = load_model('model.h5')

# Loading the models and encoders

with open('one_hot_encoding_geo.pkl', 'rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('label_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [12]:
# Example data

data =[ {
  'CreditScore': 722,
  'Geography': 'Spain',
  'Gender': 'Male',
  'Age': 38,
  'Tenure': 4,
  'Balance': 118927.66,
  'NumOfProducts': 3,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 77212.9,
  'Exited': 1
}
, {
  'CreditScore': 615,
  'Geography': 'France',
  'Gender': 'Female',
  'Age': 50,
  'Tenure': 6,
  'Balance': 37945.12,
  'NumOfProducts': 2,
  'HasCrCard': 0,
  'IsActiveMember': 1,
  'EstimatedSalary': 53211.56,
  'Exited': 0
}]

col = [
    'CreditScore',
    'Geography',
    'Gender',
    'Age',
    'Tenure',
    'Balance',
    'NumOfProducts',
    'HasCrCard',
    'IsActiveMember',
    'EstimatedSalary',
    'Exited'
]


In [13]:
temp_data = pd.DataFrame(data)
temp_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,722,Spain,Male,38,4,118927.66,3,1,1,77212.90,1
1,615,France,Female,50,6,37945.12,2,0,1,53211.56,0


In [16]:
temp_data['Gender'] = label_encoder_gender.transform(temp_data['Gender'])
temp_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,722,Spain,1,38,4,118927.66,3,1,1,77212.90,1
1,615,France,0,50,6,37945.12,2,0,1,53211.56,0


In [18]:
encoded_geo = one_hot_encoder_geo.transform(temp_data[['Geography']])
encoded_geo

c:\Users\DELL\OneDrive\Desktop\ann\env\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


array([[0., 0., 1.],
       [1., 0., 0.]])

In [19]:
features = one_hot_encoder_geo.get_feature_names_out(['Geography'])
features

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [20]:
geo_data = pd.DataFrame(encoded_geo, columns=features)
geo_data

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [21]:
temp_data = pd.concat([temp_data.drop('Geography', axis=1), geo_data], axis=1)
temp_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,722,1,38,4,118927.66,3,1,1,77212.90,1,0.0,0.0,1.0
1,615,0,50,6,37945.12,2,0,1,53211.56,0,1.0,0.0,0.0


In [23]:
X = temp_data.drop('Exited', axis=1)
y = temp_data['Exited']

temp_scaled = scaler.transform(X)

In [26]:
prediction = model.predict(temp_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [30]:
for pred in prediction:
    if pred[0]>0.5:
        print("The customer will not likely churn!!")
    else:
        print("The customer will likely churn....")

The customer will not likely churn!!
The customer will likely churn....
